<a href="https://colab.research.google.com/github/jsroa15/KKBOX/blob/main/Logs_Data_Mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Import

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
#Loading data

logs=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/KKBOX/logs.csv',nrows=30000000) #Only 30 million rows beacause of laptop memory

In [3]:
#Explore data

logs.head()

#Drop first column

logs.drop('Unnamed: 0',axis=1,inplace=True)

In [4]:
#Datset info

logs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000000 entries, 0 to 29999999
Data columns (total 9 columns):
 #   Column      Dtype  
---  ------      -----  
 0   msno        object 
 1   date        int64  
 2   num_25      int64  
 3   num_50      int64  
 4   num_75      int64  
 5   num_985     int64  
 6   num_100     int64  
 7   num_unq     int64  
 8   total_secs  float64
dtypes: float64(1), int64(7), object(1)
memory usage: 2.0+ GB


In [ ]:
#Summary statistics

logs.describe()


,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
count,3.000000e+07,3.000000e+07,3.000000e+07,3.000000e+07,3.000000e+07,3.000000e+07,3.000000e+07,3.000000e+07
mean,2.015739e+07,6.509484e+00,1.640104e+00,1.016929e+00,1.129029e+00,3.071290e+01,3.007936e+01,-1.434526e+12
std,6.181196e+03,1.450417e+01,4.268034e+00,2.177151e+00,2.923620e+00,4.277029e+01,3.375265e+01,1.152648e+14
min,2.015010e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,-9.223372e+15
25%,2.015082e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.000000e+00,8.000000e+00,1.891120e+03
50%,2.016031e+07,2.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.700000e+01,1.900000e+01,4.626872e+03
75%,2.016091e+07,7.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,3.800000e+01,4.000000e+01,1.026319e+04
max,2.017023e+07,7.112000e+03,9.070000e+02,6.440000e+02,2.417000e+03,3.469600e+04,2.767000e+03,9.223372e+15


In [ ]:
#Checking missing values

print(logs.isnull().sum())

msno          0
date          0
num_25        0
num_50        0
num_75        0
num_985       0
num_100       0
num_unq       0
total_secs    0
dtype: int64


In [ ]:
#Change Date format in Transactions Data Frame

logs.date=logs.date.astype('str')

logs['date']=pd.to_datetime(logs.date,errors='coerce')

In [ ]:
logs.head()

,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=,2015-05-13,0,0,0,0,1,1,280.335
1,rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=,2015-07-09,9,1,0,0,7,11,1658.948
2,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,2015-01-05,3,3,0,0,68,36,17364.956
3,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,2015-03-06,1,0,1,1,97,27,24667.317
4,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,2015-05-01,3,0,0,0,38,38,9649.029


In [ ]:
#Exploratory data analysis with sweetviz

#pip install sweetviz  #Install sweetviz
'''
import sweetviz as sv #Call sweetviz

report_logs=sv.analyze(logs)
report_logs.show_html('logs_EDA.html',open_browser=False)

'''

"\nimport sweetviz as sv #Call sweetviz\n\nreport_logs=sv.analyze(logs)\nreport_logs.show_html('logs_EDA.html',open_browser=False)\n\n"

In [ ]:
logs.msno.nunique()

1822947

In [ ]:
#Calculate the percentile 99.98 for every feature

columns=['num_25', 'num_50', 'num_75', 'num_985', 'num_100',
       'num_unq', 'total_secs']
for column in columns:
  print(column,str(np.percentile(logs[column],99.98)),' ',logs[logs[column]>np.percentile(logs[column],99.98)].shape[0])

num_25 256.0   5963
num_50 96.0   5906
num_75 43.0   5914
num_985 68.0   5966
num_100 496.0   5978
num_unq 339.0   5944
total_secs 101259.92242359927   6000


In [ ]:
#Deleting Outliers

temp=logs[(logs['num_25']<256)&(logs['num_50']<96)&(logs['num_75']<43)&(logs['num_985']<69)&(logs['num_100']<496)&(logs['num_unq']<464)]
print(temp.shape[0])


29972504


In [ ]:
#Create a new feature: Total_hours

temp['total_hours']=temp.total_secs/3600

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#Analizing total_hours

print(temp[temp['total_hours']<0].shape[0])

print(temp[temp['total_hours']<0].head())

4656
                                               msno  ...   total_hours
4497   8x6zMayYaSYhXQkZhD+YeBI0oNphdou+ODBpbtJpvBQ=  ... -2.562048e+12
5730   BtZTCXLQ0MvrlsosU/NsJabEZiz8JJz/rAfqjoGgxs8=  ... -2.562048e+12
11600  XoJGWnX0RN5ZUvUTG98Ldj3vlRXNQ3hhIQ1ei63oG7k=  ... -2.562048e+12
11750  NnLgT9VXBXCyhubkMQirm1Ns+xjiq+4v6zHmbaxfd+Y=  ... -2.562048e+12
12969  zCKbAfdXVRoZGSlmLPxQNwL7j9Lea1BbxdmU42SfJxc=  ... -2.562048e+12

[5 rows x 10 columns]


In [ ]:
#Amount of rows with total hours > 24
temp[temp['total_hours']>24].shape[0]

6951

In [ ]:
#Filtering the data based on total hours (Only positive and less than or equal to 24)

temp=temp[(temp['total_hours']>0)&(temp['total_hours']<=24)]

#Checking shape

temp.shape[0]

29960897

In [ ]:
temp.describe()

,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs,total_hours
count,2.996090e+07,2.996090e+07,2.996090e+07,2.996090e+07,2.996090e+07,2.996090e+07,2.996090e+07,2.996090e+07
mean,6.404721e+00,1.600706e+00,9.977453e-01,1.101384e+00,3.046175e+01,2.987853e+01,8.054167e+03,2.237269e+00
std,1.259704e+01,3.581458e+00,1.842117e+00,2.225149e+00,3.909793e+01,3.287497e+01,9.658737e+03,2.682982e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e-03,2.777778e-07
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.000000e+00,8.000000e+00,1.889828e+03,5.249522e-01
50%,2.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.700000e+01,1.900000e+01,4.620161e+03,1.283378e+00
75%,7.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,3.800000e+01,4.000000e+01,1.023421e+04,2.842835e+00
max,2.550000e+02,9.500000e+01,4.200000e+01,6.800000e+01,4.950000e+02,4.610000e+02,8.640000e+04,2.400000e+01


In [ ]:
#Defining the logs data frame grouped by user

df_logs=temp.groupby(by='msno').mean()

In [ ]:
#Exporting df_logs

df_logs.to_csv('/content/drive/MyDrive/Colab Notebooks/KKBOX/df_logs.csv',index=False)